# 변이형 오토인코더

## 데이터 load

In [ ]:
def map_image(image, label):

## 신경망 구조 정의

<img src="https://i.imgur.com/auIX6vL.png" WIDTH=600/>|

### Sampling Class

먼저 'Sampling' 클래스를 빌드합니다. 이것은 인코더 출력의 평균 (mu) 및 표준 편차 (sigma)와 함께 가우스 노이즈 입력을 제공하는 맞춤형 Keras 레이어입니다. 실제로 이 레이어의 출력은 다음 방정식으로 제공됩니다.

$$z = \mu + e^{0.5\sigma} * \epsilon  $$

여기서 $\mu$ = mean, $\sigma$ = standard deviation, $\epsilon$ = random sample

In [ ]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        # 인코더의 출력을 분해합니다.
        # 배치의 크기와 차원을 얻습니다.
        # 무작위 텐서를 생성합니다.
        # 재매개변수화 기법을 적용합니다.

### Kullback–Leibler Divergence
모델의 생성 능력을 향상 시키려면 잠재 공간에 도입된 랜덤 정규 분포를 고려해야 합니다. 이를 위해 [Kullback–Leibler Divergence](https://arxiv.org/abs/2002.07514)가 계산되어 재구성 손실에 추가됩니다. 공식은 아래 함수에서 정의됩니다.

In [ ]:
def kl_reconstruction_loss(inputs, outputs, mu, sigma):
    # KLD 계산 공식 적용
    # 배치 내 평균을 계산하고 -0.5를 곱하여 최종 KLD 손실을 계산

### VAE Model
이제 전체 VAE 모델을 정의할 수 있습니다. KL reconstruction loss를 추가하기 위해 `model.add_loss()`를 사용합니다. 이 손실을 계산하는 것은 `y_true`와 `y_pred`를 사용하지 않으므로 `model.compile()`에서 사용할 수 없습니다.

- add_loss() 메서드 : 손실이 있는 경우, 자동으로 합산되어 주 손실에 추가

In [ ]:
# 인코더의 입력 - 28x28 크기의 그레이스케일 이미지
# 플랫튼 레이어를 통해 특징 맵을 1차원 벡터로 변환
# Dense 레이어를 통해 특징을 20차원의 벡터로 변환
# 잠재 공간의 평균과 로그 분산을 나타내는 두 개의 Dense 레이어를 정의
# Sampling 레이어를 사용하여 평균과 로그 분산을 기반으로 잠재 벡터 z를 샘플링
# Sampling() 함수는 재매개변수화(reparameterization) 트릭을 구현
# 입력과 출력을 정의하여 인코더 모델을 생성합니다.
# 인코더 모델의 요약 정보를 출력합니다.

In [ ]:
# 디코더의 입력 - 잠재 공간의 차원과 일치해야 합니다.
# Dense 레이어를 통해 입력을 원본 이미지의 차원으로 변환하기 위한 벡터로 확장
# shape[1] * shape[2] * shape[3]은 원본 이미지 차원을 재구성하기 위한 총 원소 수
# Reshape 레이어를 통해 1차원 벡터를 원본 이미지의 차원과 동일한 3차원 형태로 재구성
# Conv2DTranspose 레이어를 사용하여 업샘플링과 동시에 특징 맵 학습
# 최종 Conv2DTranspose 레이어는 원본 이미지의 채널 수와 일치하는 필터 수(예: 그레이스케일 이미지의 경우 1)를 사용하며,
# sigmoid 활성화 함수를 통해 최종 이미지의 픽셀 값을 [0, 1] 범위로 조정합니다.
# 입력과 출력을 정의하여 디코더 모델을 생성

In [ ]:
# VAE 모델의 입력을 인코더의 입력으로 설정합니다. 이는 원본 이미지 데이터가 됩니다.
# 인코더를 통과시켜 평균(mu), 표준 편차(sigma), 그리고 잠재 공간의 벡터(z)를 얻습니다.
# 디코더를 사용하여 잠재 공간의 벡터(z)로부터 이미지를 재구성
# 입력과 재구성된 이미지를 사용하여 VAE 모델을 생성
# Kullback-Leibler 발산을 계산하는 함수를 사용하여 손실을 추가합니다. 이는 잠재 공간의 분포가
# 우리가 원하는 분포(예: 표준 정규 분포)에 가까워지도록 돕습니다.
# add_loss는 사용자 정의 손실을 추가하는 메서드

In [ ]:
# 학습 과정에서의 평균 손실을 추적
# 이진 교차 엔트로피 손실 함수를 사용합니다.

## 오토인코더 훈련

In [ ]:
def generate_and_save_images(decoder, epoch, step, test_input):
    # 테스트 입력에서 이미지 생성
    # 결과 플롯

In [ ]:
            # VAE 모델에 배치 입력을 공급하여 재구성된 결과를 얻음
            # 재구성 손실을 계산
            # 입력과 출력을 평탄화하여 이진 교차 엔트로피 손실 계산
            # KLD 정규화 손실을 추가
        # 그라디언트를 계산하고 가중치 업데이트
        # 손실 지표 계산
        # 매 100 스텝마다 결과 표시

## 원본 그림 재구성